# Bayesian Knowledge Tracing (initial model)

This notebook consists of a baseline BKT model as defined by Corbett & Anderson 1995. BKT aims to track student knowledge states for a given skill using underlying Hidden Markov Models, where the latent states are knowledge of a skill and observed states are correctness in answering questions related to that skill.

Each skill will be represented as a BKT model with 4 parameters:
* $p(L_0)$ or p_init_ : a priori knowledge of the skill (before any practice) 
* $p(T)$ or p_transit : probability of transitioning from the unknown to known state for that concept
* $p(G)$ or p_guess : probability of answering a question correct if the skill is in the unknown state
* $p(S)$ or p_slip : probability of answering a question wrong if the skill is in the known state

And each student will have their own parameter subset $p(L_t)^k$ for each skill $k$ as they answer $t$ questions, which will be updated at each time point (question) $t$ per skill

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import sklearn
from sklearn import metrics
import sys
sys.path.append('../')
from cf_matrix import make_confusion_matrix

## Loading data:

Loading all the dataframes here although I won't be looking at the lectures metadata for this model. Testing data will be used only for assessing model accuracy while training the models will use both the training and questions metadata.

In [5]:
# load data
data_dir = '../data/'

start = time.time()
df_train = pd.read_csv(data_dir + 'train.csv')
print('Training Data Loaded, time elapsed:', round(time.time() - start, 3), 'seconds')
df_lectures = pd.read_csv(data_dir + 'lectures.csv')
df_questions = pd.read_csv(data_dir + 'questions.csv')
df_test = pd.read_csv(data_dir + 'example_test.csv')

print('All Dataframes Loaded ------------------------------')

Training Data Loaded, time elapsed: 95.398 seconds
All Dataframes Loaded ------------------------------


### Viewing initial dataframes

In [6]:
df_train

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...,...
101230327,101230327,428564420,2147482888,3586,0,22,0,1,18000.0,True
101230328,101230328,428585000,2147482888,6341,0,23,3,1,14000.0,True
101230329,101230329,428613475,2147482888,4212,0,24,3,1,14000.0,True
101230330,101230330,428649406,2147482888,6343,0,25,1,0,22000.0,True


In [7]:
df_questions

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


### Cleaning

In [8]:
# dropping unneeded columns from train
dropcols = ['row_id', 'task_container_id', 'prior_question_elapsed_time', 'prior_question_had_explanation']
df_train.drop(dropcols, axis=1, inplace=True)

In [9]:
# since we aren't looking at lectures, I'm restricting training data to exclude lectures (content_type_id = 1)
print('Dropping {} rows...'.format(len(df_train[df_train['content_type_id'] == 1])))
df_train = df_train[df_train['content_type_id'] == 0]

Dropping 1959032 rows...


In [10]:
# can drop content_type_id now because they're all the same value (0)
df_train.drop('content_type_id', axis=1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
# dropping bundle_id and part from questions
# bundle_id represents which questions are shown together and may be used for future modifications of the model,
#  but different bundles don't seem to correspond to different tags so I won't use them here
# part corresponds to a higher level type of question ranging rom 1-5 that are too broad for this analysis
dropcols = ['bundle_id', 'part']
df_questions.drop(dropcols, axis=1, inplace=True)

In [12]:
# changing 'tags' column to a list structure so its easier to query later on and setting null value to -1
df_questions['tags'].fillna('-1', inplace=True)
df_questions['tags'] = df_questions['tags'].str.split()

### Viewing cleaned dataframes

In [13]:
df_train

,timestamp,user_id,content_id,user_answer,answered_correctly
0,0,115,5692,3,1
1,56943,115,5716,2,1
2,118363,115,128,0,1
3,131167,115,7860,0,1
4,137965,115,7922,1,1
...,...,...,...,...,...
101230327,428564420,2147482888,3586,0,1
101230328,428585000,2147482888,6341,3,1
101230329,428613475,2147482888,4212,3,1
101230330,428649406,2147482888,6343,1,0


In [14]:
df_questions

,question_id,correct_answer,tags
0,0,0,"[51, 131, 162, 38]"
1,1,1,"[131, 36, 81]"
2,2,0,"[131, 101, 162, 92]"
3,3,0,"[131, 149, 162, 29]"
4,4,3,"[131, 5, 162, 38]"
...,...,...,...
13518,13518,3,[14]
13519,13519,3,[8]
13520,13520,2,[73]
13521,13521,0,[125]


## Fitting skill parameters

There seems to be a few ways in the literature about how one can do this: EM, brute-force search, and conjugate gradient search have all been done. Here, I've chosen to use the **Empirical Probabilities** approach as defined in Hawkins, Heffernan, and Baker 2011, where they shown that this way of initializing the model parameters resulted in statistically significant increase in model performance. 

In this model the skill parameters will not change with training and will remain constant across time and students...future direction will include accounting for student-specific parameters and potentially modifying those parameters as needed for training.

### Computing knowledge sequences

The first step in the empirical probabilities approach to fitting skill parameters is to compute 'knowledge sequences' for each skill for each student. I am treating each separate 'tag' as its own skill, and questions that have multiple tags will count towards the knowledge sequences for all of the tags. 

Additionally for each skill, each student has their own knowledge sequence and all the knowledge sequences for a skill will be used to calculate that skill's BKT parameters

In [15]:
def get_knowledge_sequence(response):
    """
    Given a list of student responses to questions within a skill, this function computes the 
    knowledge sequence defined by Hawkins et al. (2011)
    This function (1) computes all knowledge permuatations of the length of the student's responses, where
    these permutations have the condition that a wrong answer cannot come after a right answer (no forgetting), 
    then (2) matches the responses to the permutation that's most similar and returns that sequence.
    If there is a tie, the average is taken
    
    Parameters
    ----------
    responses: sequence of student's answers to questions (correct/incorrect) as binary sequence
    
    Returns
    -------
    sequence: binary knowledge sequence where change from 0->1 signifies transition to learned state
    
    """
    l = len(response)
    maxs = []
    maxacc = 0
    
    def accuracy(resp, perm):
        """Get accuracy of student response and a given permutation"""
        unit = 1 / l
        acc = 0
        for i in range(l):
            if resp[i] == perm[i]:
                acc += unit
        return acc
    
    for i in range(l+1):
        perm = ([0] * (l-i)) + ([1] * i)
        if accuracy(response, perm) > maxacc:
            maxacc = accuracy(response, perm)
            maxs = [perm]
        elif accuracy(response, perm) == maxacc:
            maxs.append(perm)
    
    if len(maxs) == 1:
        return maxs[0]
    else:
        maxs = np.array(maxs)
        return list(maxs.mean(axis=0))
    

In [16]:
def get_all_sequences(df_train):
    """
    Function to get all of the knowledge sequences given the training data by going through each student
    and recording correctness to answers in each skill (tag), then calls get_knowledge_sequence to compute
    the knowledge sequences for each skill
    
    Params
    -------
    df_train : training dataframe
    
    Returns
    --------
    responses : raw correct/incorrect sequences for each skill
    sequences : dict of knowledge sequences per skill, where keys are a skill and values are a list of 
                knowledge sequences for that skill (1 per student who was tested on the skill)
    """
    print('TOTAL USERS: {}'.format(len(df_train['user_id'].unique())))
    start = time.time()
    
    # construct student responses for each tag
    responses = dict() # correctness (binary) scores
    sequences = dict() # knowledge sequences
    for n,user in enumerate(df_train['user_id'].unique()):

        # for each interaction of this user
        stuseentags = [] # tags that students has a running score for
        for i,row in df_train[df_train['user_id'] == user].sort_values(by='timestamp').iterrows():
            tags = df_questions.loc[df_questions['question_id'] == row['content_id'], 'tags'].values[0]

            for t in tags:
                responses[t] = responses.get(t, []) # initializes a list if its a new tag
                if t not in stuseentags:
                    stuseentags.append(t)
                    responses[t].append([row['answered_correctly']])
                else:
                    responses[t][-1].append(row['answered_correctly'])
                    
        for tag in stuseentags:
            sequences[tag] = sequences.get(tag, [])
            sequences[tag].append(get_knowledge_sequence(responses[tag][-1]))
        
        if n % 50 == 0:
            print('USER {} COMPLETED----------------\tTime Elapsed: {} seconds'.format(n, time.time() - start))
            start = time.time()
            #break
    
    return responses,sequences

### Fitting parameters to each skill

Given the responses and knowledge sequences, I'm going to fit the parameters for each skill using the following equations (as given by Hawkins 2011):

$$p(L_0) = \frac{\sum{K_0}}{|K_0|}$$


$$p(T) = \frac{\sum_{i!=0}{(1-K_{i-1})}K_i}{\sum_{i!=0}{(1-K_{i-1})}}$$


$$p(G) = \frac{\sum_iC_i(1-K_i)}{\sum_i (1-K_i)}$$


$$p(S) = \frac{\sum_i (1-C_i)K_i}{\sum_i K_i}$$

In [17]:
### CALL THIS FUNCTION TO FIT PARAMS, ALL OTHER FUNCS ARE ENCAPSULATED IN HERE ############
###########################################################################################
## DO NOT DO NOT DO NOT CALL THIS FUNCTION AGAIN -- PARAMS ARE STORED IN JSON FILE ######## 
######## FUNCTION MODIFIED TO RANDOML SELECT 10% OF STUDENTS TO FIT PARAMS TO #############
###########################################################################################
def fit_params_EP(df_train, fit_subset):
    """
    Wrapper function to fit parameters, calls all other functions necessary and then computes 4 parameters
    per skill using the Empirical Probabilities method (Hawkins 2011)
    
    Parameters
    ---------
    df_train : training dataframe to work with
    fit_subset : subset of training data to fit parameters to (saving computation) - must be (0.1]
    
    Returns
    --------
    params :  dict where keys are skill tags, values are a dict of the 4 parameters 
                    (p_init, p_transit, p_guess, p_slip)
    """
    
    random_users = []
    for user in df_train['user_id'].unique():
        rand = np.random.rand()
        if rand < fit_subset:
            random_users.append(user)
    df_train = df_train[df_train['user_id'].isin(random_users)]
    
    C, K = get_all_sequences(df_train)
    
    params = dict()
    for key in C.keys():
        p = dict()

        p['p_init'] = sum([i[0] for i in K[key]]) / len(K[key])

        pt_n, pt_d = 0,0
        pg_n, pg_d = 0,0
        ps_n, ps_d = 0,0

        for i in range(len(max(K[key], key=len))):
            for seq in range(len(K[key])):
                if len(K[key][seq]) <= i:
                    continue
                if i > 0:
                    pt_n += (1-K[key][seq][i-1]) * K[key][seq][i]
                    pt_d += (1-K[key][seq][i-1])
                pg_n += C[key][seq][i] * (1-K[key][seq][i])
                pg_d += 1-K[key][seq][i]
                ps_n += (1-C[key][seq][i]) * K[key][seq][i]
                ps_d += K[key][seq][i]

        p['p_transit'] = pt_n / pt_d if pt_d != 0 else 0
        p['p_guess'] = pg_n / pg_d if pg_d != 0 else 0
        p['p_slip'] = ps_n / ps_d if pg_d != 0 else 0

        params[key] = p
    
    return params

In [ ]:
# RUN THIS LATER TO STORE PARAMS (prob in datahub?)
params = fit_params_EP(df_train, fit_subset = 0.1)

# save params to json
with open('skill_params.json', 'w') as json_file:
    json.dump(params, json_file)

In [ ]:
# to get the skill parameters
f = open('skill_params.json')
params = json.load(f)

### Looking at parameter distributions

In [ ]:
df_params = pd.DataFrame(params).T
df_params

In [ ]:
# viz param dist
plt.figure(figsize=(10,6))
sns.kdeplot(data=df_params, fill=True, palette='viridis')
plt.xlabel('Probability')
plt.title('Parameter Distributions')
plt.show()

In [ ]:
# checking for model degeneracy
np.where(df_params['p_guess'] >= 0.5 | df_params['p_slip'] >= 0.5).count()

No model degenarcy (CHECK / RUN AGAIN) which is good, graph shows same pattern and initial learning is pretty normally distributed around 50% across the skills, p_transit is a little lower

## Training the Model

In training the model, each student's learning for each skill is tracked and updated as new observable states come in. This will allow for the skill-specific parameters to be tested and for correctness of a skill for a given student to be predicted as well.

Updating student skill parameters $p(L_t)^k_u$ representing the probability of skill mastery for skill $k$ and student $u$ at observation $t$ in the skill is as follows:

(1) $$p(L_1)^k_u = p(L_0)^k$$


(2) $$p(L_{t+1} | obs = correct)^k_u = \frac{p(L_t)^k_u \cdot (1 - p(S)^k)}{p(L_t)^k_u \cdot (1 - p(S)^k) + (1 - p(L_t)^k_u) \cdot p(G)^k}$$


(3) $$p(L_{t+1} | obs = wrong)^k_u = \frac{p(L_t)^k_u \cdot p(S)^k}{p(L_t)^k_u \cdot p(S)^k + (1 - p(L_t)^k_u) \cdot (1 - p(G)^k)}$$


(4) $$p(L_{t+1})^k_u = p(L_{t+1} | obs)^k_u + (1 - p(L_{t+1} | obs)^k_u) \cdot p(T)^k$$


(5) $$p(C_{t+1})^k_u = p(L_t)^k_u \cdot (1 - p(S)^k) + (1 - p(L_t)^k_u) \cdot p(G)^k$$


Where equation (1) is used as the a priori knowledge of a skill, equations (2) and (3) are conditional probabilities of student mastery of the skill given their answer (correct/incorrect) to a question of that skill, equation (4) is used to update the probability of student mastery of that skill, and equation (5) is for predictions of how the student will perform on subsequent questions within that skill

In [18]:
class BKTModel:
    
    def __init__(self, skill_params, q_tags):
        self.skill_params = skill_params
        self.users = dict() # form: {user_id : {skill: .1, skill2: .9, etc}}
        self.questions = q_tags
    
    # eq 1
    def get_prior(self, skill):
        return self.skill_params[skill]['p_init']
    
    def get_user_skill(self, user, skill):
        """
        Gets the p(L) of a user for given skill, initializes the user/skill if not already in self.users
        """
        if user not in self.users.keys():
            self.users[user] = dict()
        if skill not in self.users[user]:
            self.users[user][skill] = self.get_prior(skill)
        return self.users[user][skill]
    
    def train(self, df):
        """
        To train the model 
        
        parameters
        ----------
        df : pandas dataframe of training data where each row corresponds to an event/observation
        """
        # helper functions corresponding to above equations
        # eq 2
        def cond_correct(user, skill):
            pL = self.get_user_skill(user, skill)
            top = pL * (1 - self.skill_params[skill]['p_slip'])
            bottom = top + ((1 - pL) * self.skill_params[skill]['p_guess'])
            return top / bottom
        # eq 3
        def cond_wrong(user, skill):
            pL = self.get_user_skill(user, skill)
            top = pL * self.skill_params[skill]['p_slip']
            bottom = top + ((1 - pL) * (1 - self.skill_params[skill]['p_guess']))
            return top / bottom
        # eq 4, calls 2 or 3
        # 'correct' is bool (0=false/1=true)
        def update_knowledge(user, skill, correct):
            if correct:
                cond_pL = cond_correct(user, skill)
            else:
                cond_pL = cond_wrong(user, skill)
            self.users[user][skill] = cond_pL + ((1 - cond_pL) * self.skill_params[skill]['p_transit'])
            
        # main training loop
        print('BEGINNING TRAINING --------------------------------------------------------------')
        start = time.time()
        for i,row in df.iterrows():
            tags = self.questions[row['content_id']]
            for tag in tags:
                if tag == '-1':
                    continue
                update_knowledge(row['user_id'], tag, row['answered_correctly'])
            
            if i % 1000000 == 0:
                print('Iteration {} done -------------------- Time elapsed: {} seconds'.format(i, time.time() - start))
                start = time.time()
                
    # eq 5
    def predict(self, user, skill):
        """
        Function to predict correctness of a single user to answering a question within a skill
        """
        pL = self.get_user_skill(user, skill)
        return (pL * (1 - self.skill_params[skill]['p_slip'])) + ((1 - pL) * self.skill_params[skill]['p_guess'])
    
    def test(self, df, cutoff, confusion_mat=True, raw_probs=True):
        """
        Function to run predictions on testing data. 
        If a question has multiple tags then I take the average of those predictions
        
        parameters
        ----------
        df : pandas dataframe with testing data, must be in format where 1 row = 1 observation
        cutoff : cutoff probability to be marked as correct (ie if 0.5 then everything >= 0.5 = correct)
        confusion_mat : whether to return a confusion matrix plot, default True
        raw_probs : whether to return the raw probabilities (before casting to 0/1 based on cutoff), default True
        
        returns
        --------
        df_test : test dataframe with predicted correct/incorrect column
                    also includes raw_probability column if param is True
        accuracy : computed accuracy
        confusion : confusion matrix visualization
        """
        
        predicted = []
        raw_probability = []
        for i,row in df.iterrows():
            tags = self.questions[row['content_id']]
            tot = 0
            for tag in tags:
                tot += self.predict(row['user_id'], tag)
            raw_probability.append(tot / len(tags))
            predicted.append(int((tot/len(tags)) > cutoff))
        
        df_test = df.copy()
        df_test['predicted'] = predicted
        if raw_probs:
            df_test['raw_probability'] = raw_probability
        accuracy = sklearn.metrics.accuracy_score(df_test['answered_correctly'].values, predicted)
        
        if confusion_mat:
            confusion = sklearn.metrics.confusion_matrix(df_test['answered_correctly'].values, predicted)
            categories = ['incorrect', 'correct']
            confusion_viz = make_confusion_matrix(confusion, categories=categories)
            return df_test, accuracy, confusion_viz
        return df_test, accuracy

In [ ]:
# initialize model
q_tags = dict(zip(df_questions['question_id'], df_questions['tags']))
params = params
BKT = BKTModel(params, q_tags)

In [ ]:
# ONLY RUN ONCE AND SAVE DATA
BKT.train(df_train)

# save user params
# SPLIT INTO MULTIPLE WHEN SAVING OR EVERYTHING WILL GO TO HELL AND CRASH LIKE LAST TIME AND YOULL HAVE TO RETYPE EVERYTHING
with open('users_trained.json', 'w') as json_file:
    json.dump(BKT.users, json_file)

In [ ]:
# get user parameters again (PROB NEED TO CHANGE FOR MULTIPLE FILES)
f = open('users_trained.json')
BKT.users = json.load(f)

In [ ]:
# # SPLITTING INTO MULTIPLE FILES FOR UPLOAD
# for i,(k,v) in enumerate(BKT.users.item()):
#     if i % 59531 == 0:
#         if i != 0:
#             with open(name, 'w') as json_file:
#                 json.dump(group, json_file)
#             print('{} done'.format(i/59531))
#         group = dict()
#         name = 'users_trained_{}.json'.format(int(i/59531))
#     group[k] = v
# with open(name, 'w') as json_file:
#     json.dump(group, json_file)

## Testing predictions

I kind of miscalculated here and df_test doesn't have the true labels so I can't actually use that ... Instead I'll randomly choose 1% of training users to test on (those users' last row/observation in training)

In [ ]:
test_users = [user for user in df_train['user_id'].unique() if np.random.rand() < 0.01]
len(test_users)

In [ ]:
df_newtest = pd.DataFrame()
start = time.time()
for i,user in enumerate(test_users):
    tmp = df_train[df_train['user_id'] == user].iloc[-1:]
    df_newtest = pd.concat([df_newtest, tmp])
    if i % 100 == 0:
        print('user {} done ------ time elapsed: {} seconds'.format(i, time.time() - start))
        start = time.time()

In [ ]:
df_results, acc, confusion = BKT.test(df_newtest, cutoff=0.5)

In [ ]:
print(acc)

In [ ]:
df_results

In [ ]:
# pred correct/incorrect distributions
pred_correct = df_results[df_results['answered_correctly'] == df_results['predicted']]
pred_wrong = df_results[df_results['answered_correctly'] != df_reuslts['predicted']]

plt.scatter(pred_correct['raw_probability'].values, np.random.normal(0,2,len(pred_correct)), marker='.', color='r', label='correct')
plt.scatter(pred_wrong['raw_probability'].values, np.random.normal(0,2,len(pred_wrong)), marker='.', color='b', label='wrong')
plt.legend()
plt.show()

In [ ]:
corr = []
for i,row in df_results.iterrows():
    if row['answered_correctly'] == row['predicted']:
        corr.append('correct')
    else:
        corr.append('incorrect')
df_results['outcome'] = corr
sns.displot(data=df_results, x='raw_probability', hue='outcome', kde=True)
plt.show()

In [ ]:
print('Mean probabilities predicted correct: {}'.format(pred_correct['raw_probability'].mean()))
print('Mean probabilities predicted wrong: {}'.format(pred_wrong['raw_probability'].mean()))

In [ ]:
sklearn.metrics.roc_auc_score(df_results['answered_correctly'], df_results['predicted'])